# Rescan photos scripts

In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

cred = credentials.Certificate(r"C:\!Dev\projects\book_detection\keys\avea-biblosphere-210106-firebase-adminsdk-s3bjc-ff2c6ac275 (1).json")
firebase_admin.initialize_app(cred)

db = firestore.client()

#### Not valid photos

In [66]:
res = db.collection('photos').where(u'valid', u'==', False).stream()
finded = set()
for doc in res:
    finded.add(doc.id)
print('not valid:', len(finded))
#finded

not valid: 23


#### Photos without block 'recognition_stats'

In [67]:
res = db.collection('photos').stream()
finded = set()
for doc in res:
    data = doc.to_dict()
    if 'valid' in data and not data['valid']:
        continue
    if 'recognition_stats' in data:
        continue
    finded.add(doc.id)

print('finded', len(finded))
#finded

finded 1210


#### Rescan one photo

In [2]:
import requests
photo_id = '2KnBHTin3PzNfdAnGmPm'
url = 'https://us-central1-biblosphere-210106.cloudfunctions.net/rescan_photo'

doc = db.collection('photos').document(photo_id).get()
data = doc.to_dict()

res = requests.post(url, json={'photo_id': photo_id})
json = res.json()
print(res.status_code, res.reason)
print('duration:', json['duration'])
print('recognized:', len(json['recognized_books']))
for n, el in enumerate(json['recognized_books'], 1):
    print(f'{n}) {el}')

200 OK
duration: 0:00:37.836561
recognized: 1
1) 5936305376 Краткая история русского балета / Борис Бланков. - Санкт-Петербург : ДЕАН, 2006- (СПб. : ИПК Бионт). - 24 см.
Кн. 1. - 2006 (СПб. : ИПК Бионт). - 239 с. : ил., портр.; ISBN 5-93630-537-6 Бланков, Борис Владимирович.


In [76]:
import requests
url = 'https://us-central1-biblosphere-210106.cloudfunctions.net/rescan_photo'
batch_size = 5

res = db.collection('photos').stream()
count = 1
for doc in res:
    if count > batch_size:
        break

    data = doc.to_dict()
    if 'valid' in data and not data['valid']:
        continue
    if 'recognition_stats' in data:
        continue

    print(f'{count}) Rescaning photo: {doc.id}')

    if not data.keys() >= {'photo', 'reporter', 'bookplace', 'id', 'location', 'url'}:
        print('ERROR: photo/reporter/bookplace/id/location/url parameters missing.', doc.id, '\n')
        update = {'valid': False}
        db.collection('photos').document(doc.id).update(update)
        continue

    old_recognized = data['recognized'] if 'recognized' in data else 0
    res = requests.post(url, json={'photo_id': doc.id})
    json = res.json()
    print(f"duration: {json['duration']}, recognized: {len(json['recognized_books'])}({old_recognized})\n")

    count += 1

1) Rescaning photo: 2sbdY7e3VD8hIVcoeh1s
duration: 0:00:26.693543, recognized: 0(10)

2) Rescaning photo: 3W82C9BUmTxM7D0agG2q
duration: 0:00:23.077055, recognized: 0(0)

3) Rescaning photo: 3wMRdfSiaVY3shVKcegp
ERROR: photo/reporter/bookplace/id/location/url parameters missing. 3wMRdfSiaVY3shVKcegp 

3) Rescaning photo: 3wfqTgvP6txKNYy37DdG
ERROR: photo/reporter/bookplace/id/location/url parameters missing. 3wfqTgvP6txKNYy37DdG 

3) Rescaning photo: 43fKfeqs2JwgnvhJC5ZQ
duration: 0:00:27.626012, recognized: 6(6)

4) Rescaning photo: 46ZcPTkOGmB5tfa7YA8e
duration: 0:00:23.389939, recognized: 2(2)

5) Rescaning photo: 4EukdH5XqrEyge0bKCsl
duration: 0:00:33.688541, recognized: 6(6)



In [4]:
from PIL import Image
import requests
from io import BytesIO

photo_id = '3BG1udQExfDwZMBDK5zK'

doc = db.collection('photos').document(photo_id).get()
data = doc.to_dict()
print(data['url'])
response = requests.get(data['url'])
#response.content
img = Image.open(BytesIO(response.content))
img.show()

https://firebasestorage.googleapis.com:443/v0/b/biblosphere-210106.appspot.com/o/images%2FUyxjU2tRVB5TXY8pIDrnkLXoqVdo2:ucfsv1f%2F1627102566966.jpg?alt=media&token=feeed715-b9d6-4eac-a6a9-2acfa87c7531


In [3]:
photo_id = '0yUOgq7hjApGwORz0FDw'
doc = db.collection('photos').document(photo_id).get()
data = doc.to_dict()
data

{'location': {'geopoint': <google.cloud.firestore_v1._helpers.GeoPoint at 0x270f2032be0>,
  'geohash': 'd1u0x27h58s6'},
 'reporter': 'Syiru36ehIglQqOk2wNpgpNqG982',
 'recognition_stats': {'duration': 37499963,
  'detectron_find_books': 35,
  'known_books': 3059977,
  'record_in_stats': True,
  'date': '2021-9-23',
  'algorithm': 'Detectron build 1.0 (2021-08-26)'},
 'photo': 'images/PChIJ____b4bjoI8ReK36MRVWfzc/1611267196617.jpg',
 'id': '0yUOgq7hjApGwORz0FDw',
 'recognized': 13,
 'count': 16,
 'bookplace': 'PChIJ____b4bjoI8ReK36MRVWfzc',
 'status': 'recognized',
 'thumbnail': 'https://storage.googleapis.com/biblosphere-210106.appspot.com/thumbnails/PChIJ____b4bjoI8ReK36MRVWfzc/1611267196617.jpg',
 'total': 39,
 'url': 'https://firebasestorage.googleapis.com/v0/b/biblosphere-210106.appspot.com/o/images%2FPChIJ____b4bjoI8ReK36MRVWfzc%2F1611267196617.jpg?alt=media&token=481a9f84-32fe-4457-a1c2-119b759d5d89'}

In [75]:
photo_id = '3s6Am4UxI1EAT8JzHplv'
update = {'valid': False}
db.collection('photos').document(photo_id).update(update)

update_time {
  seconds: 1632503203
  nanos: 298959000
}